# Imports

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

# Data 
## Import relevant files

In [14]:
# Read TimeGPT forecasts
yf_df = pd.read_parquet("/Users/kristianertresvag/Documents/Handelshøyskolen BI/Master/4. semester/gittest/M-Competitions/M3-Competition/Year/M3_yearly_simple_forecasts.parquet")
qf_df = pd.read_parquet("/Users/kristianertresvag/Documents/Handelshøyskolen BI/Master/4. semester/gittest/M-Competitions/M3-Competition/Quarter/M3_quarter_simple_forecasts.parquet")
mf_df= pd.read_parquet("/Users/kristianertresvag/Documents/Handelshøyskolen BI/Master/4. semester/gittest/M-Competitions/M3-Competition/Month/M3_month_simple_forecasts.parquet")

# Read excel file with results from other methods
xls = pd.ExcelFile("/Users/kristianertresvag/Documents/Handelshøyskolen BI/Master/4. semester/gittest/M-Competitions/M3-Competition/M3Forecast.xls")


        Method  NF  Series ForecastPeriod    value
66066     AAM1   8     646              1  5588.90
136500    AAM1   8     646              2  5666.90
206934    AAM1   8     646              3  5745.40
277368    AAM1   8     646              4  5824.40
347802    AAM1   8     646              5  5903.90
...        ...  ..     ...            ...      ...
85448   WINTER   8    3003              2  3427.91
155882  WINTER   8    3003              3  3393.80
226316  WINTER   8    3003              4  3359.70
296750  WINTER   8    3003              5  3325.60
367184  WINTER   8    3003              6  3291.50

[422604 rows x 5 columns]


## Reformat TimeGPT forecasts

In [48]:
TimeGPT_forecasts = yf_df.append(qf_df)
TimeGPT_forecasts = TimeGPT_forecasts.append(TimeGPT_forecasts)


TimeGPT_forecasts['Method'] = 'TimeGPT'
TimeGPT_forecasts['ForecastPeriod'] = TimeGPT_forecasts.groupby('Series').cumcount() + 1
TimeGPT_forecasts['NF'] = TimeGPT_forecasts.groupby('Series')['ForecastPeriod'].transform('max')
TimeGPT_forecasts = TimeGPT_forecasts.rename(columns={'TimeGPT': 'value'})

TimeGPT_forecasts = TimeGPT_forecasts[['Method', 'NF', 'Series', 'ForecastPeriod', 'value']]

TimeGPT_forecasts

/var/folders/h6/pplgpj2536x9l5zs7z8vmzdm0000gn/T/ipykernel_43777/1482941596.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  TimeGPT_forecasts = yf_df.append(qf_df)
/var/folders/h6/pplgpj2536x9l5zs7z8vmzdm0000gn/T/ipykernel_43777/1482941596.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  TimeGPT_forecasts = TimeGPT_forecasts.append(TimeGPT_forecasts)


,Method,NF,Series,ForecastPeriod,value
0,TimeGPT,12,1,1,5121.754395
1,TimeGPT,12,1,2,4974.531738
2,TimeGPT,12,1,3,4750.660156
3,TimeGPT,12,1,4,4366.261719
4,TimeGPT,12,1,5,3914.425781
...,...,...,...,...,...
6035,TimeGPT,16,1401,12,4579.938477
6036,TimeGPT,16,1401,13,4569.582031
6037,TimeGPT,16,1401,14,4653.845703
6038,TimeGPT,16,1401,15,4854.271484


## Format other forecasts to dataframe

In [50]:
# Read other forecasts
dfs = []

for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
    df['Method'] = sheet_name
    dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
results_df = pd.concat(dfs, ignore_index=True)
results_df.rename(columns = {0 : 'Series', 1 : 'NF'}, inplace = True)

# Change series to int in the same fashion as the other dataframes
results_df['Series'] = results_df['Series'].str.replace('N', '')
results_df['Series'] = results_df['Series'].str.replace(' ', '')
results_df['Series'] = results_df['Series'].astype(int)

# Change position of sheet column
cols = results_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
results_df = results_df[cols]

# Rename columns so that 1 is the first forecast period
results_df.rename(columns=lambda x: x - 1 if isinstance(x, int) else x, inplace=True)

# Melt the dataframe i.e. unpivot
other_forecasts = pd.melt(results_df, id_vars=['Method', 'NF' , 'Series'], value_name='value', var_name='ForecastPeriod')   

# Sort the dataframe
other_forecasts = other_forecasts.sort_values(by=['Method', 'Series', 'ForecastPeriod']).copy()

# Remove NaN
other_forecasts = df_long.dropna(subset='value')

# Display the combined DataFrame
print(other_forecasts)

        Method  NF  Series ForecastPeriod    value
66066     AAM1   8     646              1  5588.90
136500    AAM1   8     646              2  5666.90
206934    AAM1   8     646              3  5745.40
277368    AAM1   8     646              4  5824.40
347802    AAM1   8     646              5  5903.90
...        ...  ..     ...            ...      ...
226316  WINTER   8    3003              4  3359.70
296750  WINTER   8    3003              5  3325.60
367184  WINTER   8    3003              6  3291.50
437618  WINTER   8    3003              7  3257.40
508052  WINTER   8    3003              8  3223.29

[877812 rows x 5 columns]


## Grouping the dataframes

should add Method Category and Series Category

In [53]:
results_df = pd.concat([TimeGPT_forecasts, other_forecasts], ignore_index=True)


## Visuals

In [55]:
plt.figure(figsize=(10, 6))

# We loop over each group defined by unique combinations of 'Method' and 'Series'
for (method, series), results_df in results_df.groupby(['Method', 'Series']):
    plt.plot(results_df['ForecastPeriod'], results_df['value'], marker='o', label=f'{method} - Series {series}')

plt.xlabel('Forecast Period')
plt.ylabel('Value')
plt.title('Forecast Values Over Time by Method and Series')
plt.legend()
plt.grid(True)
plt.show()

/Users/kristianertresvag/opt/anaconda3/lib/python3.9/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


ValueError: Image size of 846x1504595 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1000x600 with 1 Axes>